In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
#import csvs
monday = pd.read_csv('./data/monday.csv', sep=';')
tuesday = pd.read_csv('./data/tuesday.csv', sep=';')
wednesday = pd.read_csv('./data/wednesday.csv', sep=';')
thursday = pd.read_csv('./data/thursday.csv', sep=';')
friday = pd.read_csv('./data/friday.csv', sep=';')

days_data = [monday, tuesday, wednesday, thursday, friday]

In [4]:
#add days for the merge
monday['day'] = 'monday'
tuesday['day'] = 'tuesday'
wednesday['day'] = 'wednesday'
thursday['day'] = 'thursday'
friday['day'] = 'friday'

In [5]:
#this is to get the day timestamp
for day in days_data:
    day.timestamp = pd.to_datetime(day.timestamp)
    day.set_index('timestamp', inplace=True)

In [6]:
#for the customers that didn't leave, append a checkout
def get_non_checkout(df):
    non_checkout_customers = df['customer_no'].max()
    checkout_customers = []
    for c_id in range(non_checkout_customers):
        if not 'checkout' in df[df['customer_no'] == c_id+1]['location'].values:
            checkout_customers.append(c_id+1)
    return checkout_customers

In [7]:
#add the day for those
customers_to_checkout = []
for day in days_data:
    customers_to_checkout.append(get_non_checkout(day))

In [8]:
#this adds the checkout time to the non checkout customers
def add_checkout(df, customer_nos, date, day):
    df_fill = pd.DataFrame()
    for p_id in customer_nos:
        df_tmp = pd.DataFrame(data=[[p_id, 'checkout', day]], index=[pd.to_datetime(f'2019-09-{date} 21:59:00')], columns=['customer_no', 'location', 'day'])
        df = pd.concat([df, df_tmp])
    return df

In [9]:
filled_data = []
weekdays = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday']
date = ['02', '03', '04', '05', '06']
for i, day in enumerate(days_data):
    filled_data.append(add_checkout(day, customers_to_checkout[i], date[i], weekdays[i]))

In [10]:
#concat the 5 now corrected with all checkouts for all customers into 1 df
total = pd.concat(filled_data)

In [11]:
total

,customer_no,location,day
2019-09-02 07:03:00,1,dairy,monday
2019-09-02 07:03:00,2,dairy,monday
2019-09-02 07:04:00,3,dairy,monday
2019-09-02 07:04:00,4,dairy,monday
2019-09-02 07:04:00,5,spices,monday
...,...,...,...
2019-09-06 21:59:00,1503,checkout,friday
2019-09-06 21:59:00,1505,checkout,friday
2019-09-06 21:59:00,1506,checkout,friday
2019-09-06 21:59:00,1509,checkout,friday


In [12]:
#if we want to use minutes, we need to forward fill to it
perminute = total.groupby(['customer_no', 'day']).resample('1Min').ffill()

In [13]:
perminute

customer_no  location       day
customer_no day                                                          
1           friday   2019-09-06 07:00:00            1     dairy    friday
                     2019-09-06 07:01:00            1     dairy    friday
                     2019-09-06 07:02:00            1     dairy    friday
                     2019-09-06 07:03:00            1     dairy    friday
                     2019-09-06 07:04:00            1    spices    friday
...                                               ...       ...       ...
1533        thursday 2019-09-05 21:59:00         1533  checkout  thursday
1534        thursday 2019-09-05 21:48:00         1534     fruit  thursday
                     2019-09-05 21:49:00         1534  checkout  thursday
1535        thursday 2019-09-05 21:48:00         1535    spices  thursday
                     2019-09-05 21:49:00         1535  checkout  thursday

[54192 rows x 3 columns]

In [14]:
#Shift the `location` column to get a `from` column that describes the moving
perminute['from'] = perminute['location'].shift(1)

In [15]:
#fill the 1shift nan
perminute.fillna('checkout', inplace=True)

In [16]:
#crosstab for the markov?
tm = pd.crosstab(perminute['from'], perminute['location'], normalize='index')

In [17]:
tm

location,checkout,dairy,drinks,fruit,spices
from,,,,,
checkout,0.000000,0.287576,0.153526,0.377435,0.181464
dairy,0.102697,0.738706,0.058134,0.049478,0.050986
drinks,0.215334,0.010880,0.599199,0.087755,0.086832
fruit,0.200564,0.095428,0.054564,0.599029,0.050415
spices,0.149613,0.191839,0.161948,0.090305,0.406294
